Code pour projet crypto

Import des librairies

In [1]:
import hashlib
import random 
import gmpy2 as gm
import math
import time
import base64
import os, binascii
from backports.pbkdf2 import pbkdf2_hmac
import pyaes, pbkdf2, secrets


## Exercice 1

### Partie 1

In [2]:

def trouver_chaine(base, zeros_cibles):
        compteur = 0
        while True:
            chaine_test = base + str(compteur)
            hash_resultat = sha256_hash(chaine_test)
            if hash_resultat.endswith('0' * zeros_cibles):
                return chaine_test, compteur, hash_resultat
            compteur += 1

def sha256_hash(chaine):
        return hashlib.sha256(chaine.encode()).hexdigest()
        
def trouver_chaine_et_stats(base, zeros_cibles, iterations_stat=5):
    

    # Première partie : Trouver la chaîne
    chaine_trouvee, compteur_trouve, hash_trouve = trouver_chaine(base, zeros_cibles)

    # Deuxième partie : Statistiques de temps
    temps_n = []
    temps_n_plus_1 = []
    
    for _ in range(iterations_stat):
        debut = time.time()
        trouver_chaine(base, zeros_cibles)
        fin = time.time()
        temps_n.append(fin - debut)

        debut = time.time()
        trouver_chaine(base, zeros_cibles + 1)
        fin = time.time()
        temps_n_plus_1.append(fin - debut)

    temps_moyen_n = sum(temps_n) / len(temps_n)
    temps_moyen_n_plus_1 = sum(temps_n_plus_1) / len(temps_n_plus_1)

    rapport_temps = temps_moyen_n_plus_1 / temps_moyen_n

    return {
        "chaine_trouvee": chaine_trouvee,
        "hash_trouve": hash_trouve,
        "compteur_trouve": compteur_trouve,
        "temps_moyen_n": temps_moyen_n,
        "temps_moyen_n_plus_1": temps_moyen_n_plus_1,
        "rapport_temps": rapport_temps
    }


base_chaine = "Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6"
zeros_cibles = 4 # Nombre de zéros à la fin du hash
resultats = trouver_chaine_et_stats(base_chaine, zeros_cibles)
resultats


{'chaine_trouvee': 'Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA677031',
 'hash_trouve': '31e6f8e6377ad1c9895ef7c22b0ef9a252c0f0ec9d062178c58b416bcb470000',
 'compteur_trouve': 77031,
 'temps_moyen_n': 0.1403665065765381,
 'temps_moyen_n_plus_1': 0.42521376609802247,
 'rapport_temps': 3.029310741349574}

## Exercice 2

In [3]:
with open("format.txt", "r",encoding="utf-8") as file:
    text = str(file.read())
text

"Le marché des sneakers en France a connu une croissance exponentielle au cours des dernières années, devenant un phénomène de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumière ses dynamiques économiques, culturelles et sociales.\nD'un point de vue économique, le commerce de sneakers en France est stimulé par une demande croissante pour des modèles rares et exclusifs. \nLes éditions limitées et les collaborations entre marques de renom et designers célèbres créent une urgence d'achat, poussant les prix sur le marché secondaire à atteindre des sommets vertigineux. \nCette situation a engendré l'émergence d'une véritable économie parallèle, où les sneakers sont non seulement des objets de mode, mais également des actifs spéculatifs.\n\nSur le plan culturel, les sneakers occupent une place prépondérante dans l'univers de la mode et de la 

In [4]:
password = "123456789"
passwordSalt = os.urandom(16)
key = pbkdf2.PBKDF2(password, passwordSalt).read(32)
iv = secrets.randbits(256)
#On est obliger de recrée un mot de pass unique pour chiffrer le texte
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
ciphertext = aes.encrypt(text)

In [5]:
print(aes.decrypt(ciphertext))

b'&\x96jO\x17k\rvG\xabA_H\x99\xf0%v\x8cQ\xd5\xea\x17\x15\xe9\xec\xcf\xf3\xa5\x17\xc0.\x17(\xde\xd09\xd0\x13\xf4\x1f\xa83j\x7f\xa2=\x81\x91\xd5JA\xc1\xbd}kw:j\xee\x8d\x15\x99\x82\x1e\xf4\xe4\xa9\x0f\x0f\xb2\xf57\xae\xf9I(\x1d\xad8\x9ce\xd8\'V\x84\x1d\xe0:\xb7\xa5\x10\xda\xc4\xb8\xff_xN*\xf4\xdd\xc7\x1dC\x12\xdc\x9b\x98<\xbdm\xec\xa2$wl\xee\x8fC\xe2O:\xdaQ\xe2\xf6\x9f\x91\xe1\xec\x08\x97\x8b?\xef\xae\x8b\x8a\x97F\xaf\xfd\xda\x9f\xdf\x03\x0b\xc7\xc2\x10\xcc2-\x1a\x8d\xd1\x06\x12h\x8d=(>\xdb\xb4SO\xefv~{\xb80\xdewN\r\xaf\xebC\x8e\xb4t\xc8\xc2\xc1\xa4sl<4\x95\xde[;b\x9e\x05\xd6H\x85r\xd2\xc7B\xd9\xec4\xa6\x11\xfa~\xce\x15\x89\xd4U\x0eJw\x0c$\xf0\xd2\x0c\xa9\x1b\xfb3\xc4\x1b\xa9\xa2\xda\xc9c#^\xd9\x0f\xcf\xad\xab1\xf8\xbf8\x86\xc7_X\x90\xdfc\xf5\xac\xe5\x99\x86`G\xe2xL3\x9c\x9b\x96\xb0\xff\xb5\xed\xc8\xae+q\xd1\xc2\xd1\xd86\x9e8\xc0\xc3\x1bK\x92\xb0\xa5o@\x0c\x9ef\x91\xc0k\x17>\x8e\xfd\xcf#\xfc\xb4.f\x9b\x99\x0b\x08\x914\x96:\x04\xf8\xeb\x14R\xe9\x16\x17\xedO\x88\x97\xcet\x11\xdbv\xaf;B\xce@

In [6]:
print(pyaes.Counter(iv))
#Et du coup de le regénéré
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
print(aes.decrypt(ciphertext))

b"Le march\xe9 des sneakers en France a connu une croissance exponentielle au cours des derni\xe8res ann\xe9es, devenant un ph\xe9nom\xe8ne de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumi\xe8re ses dynamiques \xe9conomiques, culturelles et sociales.\nD'un point de vue \xe9conomique, le commerce de sneakers en France est stimul\xe9 par une demande croissante pour des mod\xe8les rares et exclusifs. \nLes \xe9ditions limit\xe9es et les collaborations entre marques de renom et designers c\xe9l\xe8bres cr\xe9ent une urgence d'achat, poussant les prix sur le march\xe9 secondaire \xe0 atteindre des sommets vertigineux. \nCette situation a engendr\xe9 l'\xe9mergence d'une v\xe9ritable \xe9conomie parall\xe8le, o\xf9 les sneakers sont non seulement des objets de mode, mais \xe9galement des actifs sp\xe9culatifs.\n\nSur le plan culturel, les s

In [7]:
p = 7946851324679854613245823
g = 5
A = 7579501795988122393422986
## Elle publie A
password = "35426517"
IV = "756431" 
N = int(password + "0000" + IV) 

b = 123456789  

# Calcul de B et C pour ElGamal
B = pow(g, b, p)
K = pow(A, b, p)  # Clé secrète partagée
C = (K * N) % p  # Chiffrement de N avec la clé secrète partagée

B, C

(387225946877039798412155, 5883019579684883513940408)

## Exercice 3

#### 1. Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).

In [8]:
# Fonction pour calculer l'inverse modulo
def inverse_modulo(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

# Fonction pour calculer la signature
def sign_elgamal(m, x, p, g):
    import random
    # Sélection d'un nombre aléatoire k
    k = random.randint(2, p - 2)
    while pow(k, -1, p - 1) == 0:  # Assure que k est premier avec p-1
        k = random.randint(2, p - 2)
    r = pow(g, k, p)
    k_inv = inverse_modulo(k, p - 1)
    s = (m - x * r) * k_inv % (p - 1)
    return r, s

# Fonction pour vérifier la signature
def verify_elgamal(m, r, s, y, p, g):
    v1 = pow(g, m, p)
    v2 = (pow(y, r, p) * pow(r, s, p)) % p
    return v1 == v2

# Paramètres ElGamal
p = 257
g = 3
x = 97

# Clé publique
y = pow(g, x, p)

# Message à signer
m = 123

# Signature
r, s = sign_elgamal(m, x, p, g)

# Vérification de la signature
valid = verify_elgamal(m, r, s, y, p, g)

print("Signature valide :", valid)


Signature valide : True


#### 2. Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

Supposons que nous ayons un fichier texte appelé "format.txt" contenant le message à signer.



**a) Génération des clés RSA :**
Tout d'abord, nous devons générer une paire de clés RSA privée/publique à l'aide d'OpenSSL.

In [10]:
#copy paste in a terminal
!openssl genpkey -algorithm RSA -out private_key.pem
!openssl rsa -pubout -in private_key.pem -out public_key.pem

.........+....+.....+...............+.+..+++++++++++++++++++++++++++++++++++++++*.....+...+......+.+........+.......+..............+....+..+.+..+.+......+...+......+..+...+.......+.....+.......+..+++++++++++++++++++++++++++++++++++++++*.+..+...................+......+........+......+.+........+...+....+......+.........+......+.....+...+....+...+.....+...+.......+........+....+............+...+..................+...+..+.......+..+.+...+.....+................+...+.....+......+.+...+..+................+...+..+....+........+.+.........+........+....+.....+...+.+...........................+.....+...+......+......+.......+......+......+.....+...+.+.........+........+...+......++++++
.+...........+...+.+......+...+..+...+.......+......+.....+++++++++++++++++++++++++++++++++++++++*........+++++++++++++++++++++++++++++++++++++++*....+....+.....+.+............+..+......+......+.+..+...+....+......+.....+....+.........+..+...+......+....+..+...............+......................+..+..............

Ces commandes génèrent une clé privée private_key.pem et une clé publique correspondante `public_key.pem`.



**b) Signature du message :**

Ensuite, nous signons le message avec notre clé privée.

In [11]:
#copy paste in a terminal
!openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt

Cela crée une signature du message dans le fichier signature.bin.


**c) Vérification de la signature :**

Pour vérifier la signature avec la clé publique correspondante, nous utilisons :

In [12]:
#copy paste in a terminal
!openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

Verified OK


Si la vérification réussit, OpenSSL affichera "Verified OK".


Cet exemple montre comment utiliser OpenSSL pour générer une paire de clés RSA, signer un message avec la clé privée et vérifier la signature avec la clé publique. Vous pouvez répéter ces étapes dans un terminal pour expérimenter avec vos propres messages et clés.

In [13]:
!openssl genpkey -algorithm RSA -out private_key.pem
!openssl rsa -pubout -in private_key.pem -out public_key.pem
!openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt       
!openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

.....+++++++++++++++++++++++++++++++++++++++*.....+...+......+++++++++++++++++++++++++++++++++++++++*...+.+........+.......+..+.......+.........+.....+......+.+..+.+.........+.....+......+....+...........+......+....+.....+......+.........+.+......+.....+....+...+...+..............+.+.....+....+..................+...+........+......+...+...............+............+.+..+.......++++++
......+...+...........+...+.+++++++++++++++++++++++++++++++++++++++*.+.+.................+..........+..............+.+..+...+.......+..+...+.+..+++++++++++++++++++++++++++++++++++++++*..+....+......+..+......+.......+..+...+...+...................+...............+..+.+............+........+...++++++


writing RSA key


Verified OK
